# Notebook for the dataset quality improvement experiment

## TOC:
### Set up
* [Imports](#imports)
* [Load in data](#load_data)
* [Initializations](#initializations)
### Errors
* [Missing values](#mvs)
* [Duplicate rows](#duplicate_rows)
* [Duplicate attributes](#duplicate_cols)
* [Outlier values](#outlier_vals)
* [Outlier instances](#outlier_rows)
* [Cryptic attribute names](#cryptic)
* [Single value attributes](#svs)
* [Mixed data types](#mixed)
* [Mislabeled labels](#mislabels)
* [String mismatches](#mismatches)

## All imports necessary to run the notebook <a class="anchor" id="imports"></a>

In [1]:
#All imports
import pandas as pd
import numpy as np
import json
import re
import nltk
import wordninja
from typing import List, Tuple, Dict
from sklearn.metrics import classification_report
from nltk.stem.wordnet import WordNetLemmatizer
import random
from openai import OpenAI
import time
import copy
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import pFAHES.common as common
import pFAHES.patterns as patterns
import pFAHES.DV_Detector as DV_Detector
import pFAHES.RandDMVD as RandDMVD
import pFAHES.OD as OD
from statistics import mean, median, mode
import copy
from sortinghatinf import get_sortinghat_types
from numpy import percentile
import tiktoken
from deepchecks.tabular.checks import MixedDataTypes
from deepchecks.tabular import Dataset
from deepchecks.tabular.checks import StringMismatch
from deepchecks.tabular.checks import SpecialCharacters
from deepchecks.tabular.checks import OutlierSampleDetection
from deepchecks.tabular.checks import MixedNulls
import math
from PyNomaly import loop
import cleanlab
import pickle
import pandas as pd
import openml
import math
from sklearn.metrics import (f1_score, accuracy_score, mean_squared_error, mean_absolute_error,
                             roc_auc_score, precision_score, recall_score, r2_score)
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import cross_validate

C:\Users\sande\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\sande\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\sande\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Load in the synthetic test data <a class="anchor" id="load_data"></a>

In [2]:
#Synthetic test data
df_synthetic = pd.read_excel("data/synthetic_data.xlsx")
df_synthetic.head()

,age,workclass,fnlwgt,education,gender,native-country,income,high_label_correlation,redundant SV column,duplicate column,mixed_data_types
0,31,Private,121321,?,Male,United-States,<=50K,average salary,this_is_the_same,31,3
1,36,Private,73023,HS-grad,Male,$$$$$,<=50K,average salary,this_is_the_same,36,MD2
2,20,Private,194686,Some-college,Male,United-States,<=50K,average salary,this_is_the_same,20,2
3,24,Private,72887,HS-grad,Male,United-States,<=50K,average salary,this_is_the_same,24,MD2
4,44,VARiation!,282192,?,Male,United-States,<=50K,CHANGED VALUE,this_is_the_same,44,3


## Initializations <a class="anchor" id="initializations"></a>

In [3]:
feature_types = get_sortinghat_types(df_synthetic)
feature_types_dct = {col:feat for col, feat in zip(df_synthetic.columns, feature_types)}
token_encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
target = "income"

## Missing values <a class="anchor" id="mvs"></a>

### Classes and help functions

In [4]:
def categorical_to_label(df, encode_nans=True):
    """ Converts categorical columns in a dataframe to labels (numerical)
    input:
        df: Pandas DataFrame in which the categorical features will be converted to labels
        encode_nans: True, if the NaNs should be encoded as a label as well (needed for most
        imputation techniques because do not work if there are NaNs in the non-target columns). False, 
        if the NaNs should not be encoded as a label (e.g. for KNN imputation)
    output:
        df_copy: Copy of the original df, but now containing labels for the categorical columns
        dct: A dictionary containing the column names (of categorical columns) as keys and the 
        LabelEncoder used for that column as value. This is needed to convert the labels back to 
        the original categorical values
    """
    dct = dict()
    df_copy = copy.deepcopy(df)
    if encode_nans:
        for col in df_copy.columns:
            if df_copy[col].dtype == 'object' or df_copy[col].dtype == 'category' or df_copy[col].dtype == 'bool':
                df_copy[col] = df_copy[col].astype(str)
                encoder = LabelEncoder()
                df_copy[col] = encoder.fit_transform(df_copy[col])
                dct[col] = encoder
    else:
        for col in df_copy.columns:
            if df_copy[col].dtype == 'object':
                nan_mask = df_copy[col].isnull()
                nan_rows = df_copy[col][nan_mask]
                cat_rows = df_copy[col][~nan_mask].astype(str)
                encoder = LabelEncoder()
                cat_as_label_rows = encoder.fit_transform(cat_rows)
                df_copy[col][~nan_mask] = cat_as_label_rows
                dct[col] = encoder
    return df_copy, dct

In [5]:
def ml_imputation(df, col, method):
    """ Imputation of a column with a Machine Learning (ML) technique
    input:
        df: Pandas DataFrame containing a column on which ML-imputation will be applied
        col: String of the column name on which the ML-imputation will be applied
        method: String of the ML-imputation method that is selected to impute the MVs
        in the selected column. Could be either: MLP, CART or RF.
    output:
        curr_col: Pandas Series of the ML-imputed column
    """
    if df[col].isna().any():
        curr_col = df[col]
        # Transform categorical string attributes in dataframe to categorical labels
        temp_df, encoder = categorical_to_label(df)        

        # Mask over the NaN values in the to be imputed column
        nan_mask = curr_col.isnull()

        # We use all other attributes (X) to predict the missing values of the to be imputed column (y)
        X = temp_df.loc[:, temp_df.columns != col]
        y = curr_col

        # We split the data based on the indices for which y is a NaN
        X_test = X[nan_mask]
        X_train = X[~nan_mask]
        y_test = y[nan_mask]
        y_train = y[~nan_mask]

        if method == 'MLP':
            model = MLPClassifier(solver='lbfgs', alpha=1e-5,
                         hidden_layer_sizes=(100,), random_state=1)
        elif method == 'CART':
            model = DecisionTreeClassifier()
        elif method == 'RF':
            model = RandomForestClassifier(max_depth=2, random_state=0)
        elif method == 'KNN_ALT':
            model = KNeighborsClassifier()
        else:
            print(f"Method {method} is not included as a ML-imputation technique")
        model.fit(X_train, y_train)
        pred_values = model.predict(X_test)

        # Map the indices to the predicted values in order to impute the values into the column
        indices = [idx for idx in X_test.index]
        mapping = {indices[i]:pred_values[i] for i in range(len(indices))}
        imputed_col = copy.deepcopy(curr_col)
        for i, j in mapping.items():
            imputed_col.loc[i] = j
        return imputed_col
    else:
        return df[col]

In [6]:
def mv_method(df, col, method):
    """ Applies the selected imputation method on the selected column
    input:
        df: Pandas DataFrame containing a column on which imputation will be applied
        col: String of the column name on which the imputation will be applied
        method: String of the imputation method that is selected to impute the MVs
        in the selected column. Could be either: Mode, MLP, CART, RF, KNN, Delete record
        and Do not impute (and Mean and Median too for numerical columns)
    output:
        curr_col: Pandas Series of the imputed column
    """
    curr_col = df[col]
    if method == "mode":
        col_mode = df[col].value_counts().idxmax()
        imputed_col = curr_col.fillna(col_mode)
    elif method == 'mean':
        imputed_col = curr_col.fillna(mean(curr_col))
    elif method == 'median':
        imputed_col = curr_col.fillna(median(curr_col))
    elif method == 'RF' or method == "MLP" or method == 'CART' or method == 'KNN_ALT':
        imputed_col = ml_imputation(df, col, method)
    elif method == 'Do not impute':
        imputed_col = curr_col
    elif method == 'Delete record':
        mv_records = df[df[col].isna() == True].index
        cleaned_df = df.drop(mv_records)
        return cleaned_df
    else:
        print(f"Method {method} is not known")
        
    return imputed_col

### Detect

In [7]:
def detect_mv(df):
    """ Detects whether there are missing values in a column
    input: 
        series: Pandas Series of a column in a dataframe
    output: 
        boolean: True if missing values in the column, False if not
    """
    boolean = df.isnull().any()
    nan_cols = df.columns[boolean].to_list()
    mv_list = []
    for col in nan_cols:
        freq = len(df[df[col].isna()])
        mv_list.append([col, 'nan', freq])
    return mv_list

In [8]:
def detect_dmv(df, tool_id="5"):
    """ Detects disguised missing values (DMVs) in a dataframe
    input: 
        df: Pandas DataFrame that will be checked on DMVs
        tool_id: String of 1, 2, 3, 4 or 5, which determines what type of DMVs
        will be checked for. Default=5; pattern discovery + numerical outlier detection
    output:
        output_list: List containing the DMVs detected stored as lists within that list
        in the format: Column name, DMV, Frequency (of DMV), Detection tool (that found the DMV)
    """
    
    df = df.astype(str)
    sus_dis_values = []

    if tool_id == '1':
        sus_dis_values, ptrns = patterns.find_all_patterns(df, sus_dis_values)
        sus_dis_values = DV_Detector.check_non_conforming_patterns(df, sus_dis_values)
    elif tool_id == '2':
        sus_dis_values = RandDMVD.find_disguised_values(df, sus_dis_values)
    elif tool_id == '3':
        sus_dis_values = OD.detect_outliers(df, sus_dis_values)
    elif tool_id == '4':
        sus_dis_values, ptrns = patterns.find_all_patterns(df, sus_dis_values)
        sus_dis_values = DV_Detector.check_non_conforming_patterns(df, sus_dis_values)
        sus_dis_values = RandDMVD.find_disguised_values(df, sus_dis_values)
        sus_dis_values = OD.detect_outliers(df, sus_dis_values)
    elif tool_id == '5':
        sus_dis_values, ptrns = patterns.find_all_patterns(df, sus_dis_values)
        sus_dis_values = DV_Detector.check_non_conforming_patterns(df, sus_dis_values)
        sus_dis_values = OD.detect_outliers(df, sus_dis_values)
    else:
        print("Unkown option ..",tool_id)
    
    output_str = "Attribute Name, Value, Frequency, Detection Tool \n"
    output_list = []
    for sus_dis in sus_dis_values:
        output_str = output_str + f"{sus_dis.attr_name}, {sus_dis.value}, {sus_dis.frequency}, {sus_dis.tool_name}\n"
        output_list.append([sus_dis.attr_name,sus_dis.value,sus_dis.frequency])
    
    return output_list

In [9]:
def detect_spec_chars(df):
    ''' Detects special characters using Deepchecks'''
    spec_mvs = []
    check = SpecialCharacters(n_most_common=10000, n_top_columns=10000).run(df)
    if check.display:
        result = check.display[1]
        print(result)
        for idx, specs in zip(result.index, result['Most Common Special-Only Samples']):
            for spec in specs:
                freq = len(df[df[idx] == spec])
                spec_mvs.append([idx, spec, freq])
    return spec_mvs

In [10]:
def detect_mixed_nulls(df):
    ''' Dects mixed nulls using deepchecks'''
    mixed_nulls = []
    check = MixedNulls(n_top_columns=10000).run(df)
    if check.display:
        result = check.display[1].reset_index()
        for i in range(len(result)):
            if not result['Value'][i].endswith('.nan') and result['Value'][i] != 'None':
                col = result.loc[i, 'Column Name']
                val = result.loc[i, 'Value']
                freq = result.loc[i, 'Count']
                mixed_nulls.append([col, val, freq])
    return mixed_nulls

### Correct

In [11]:
def impute_mv(df, feature_types):
    ''' Imputes missing values according to the feature types, dataset size and missingness ratio
    '''
    df_copy = copy.deepcopy(df).astype(str)
    
    # Determine imputation method
    feature_type_set = set([typ for typ in feature_types if typ == 'numeric' or typ == 'categorical'])
    dataset_size = df_copy.size
    dtypes_dct = {col:df.dtypes[col] for col in df}
    dmvs = detect_dmv(df_copy)
    mvs = detect_mv(df)
    spec_chars = detect_spec_chars(df)
    mixed_nulls = detect_mixed_nulls(df)
    all_mvs = dmvs + mvs + spec_chars + mixed_nulls
    unique_mvs = []
    nan_cols = []
    for mv in all_mvs:
        if mv not in unique_mvs:
            unique_mvs.append(mv)
        if mv[0] not in nan_cols:
            nan_cols.append(mv[0])
    print("Detected Missing Values: ", unique_mvs)
    for mv in unique_mvs:
        indices = df_copy[df_copy[mv[0]] == mv[1]].index
        df_copy.loc[indices, mv[0]] = np.nan
    
    print(f"Number of feature types: {len(feature_type_set)}")
    if len(feature_type_set) == 1:
        #not mixed data
        data_type = list(feature_type_set)[0]
        print(f"Data type: {data_type}")
        print(f"Dataset size: {dataset_size}")
        if data_type == 'numeric':
            if dataset_size < 10000:
                method = "MLP"
            else:
                method = "CART"
        else:
            if dataset_size < 10000:
                method = "RF"
            else:
                method = "CART"
    else:
        #mixed data
        nan_values = df_copy.isnull().sum().sum()
        missingness_ratio = nan_values / dataset_size
        print(f"MR: {missingness_ratio}")
        if missingness_ratio <= 0.1:
            method = "KNN"
        else:
            print(f"Dataset size: {dataset_size}")
            if dataset_size < 10000:
                method = "RF"
            else:
                method = "CART"
    print(f"Method: {method}")
            
    if method == "KNN":
        labeled_df, dct_encoder = categorical_to_label(df_copy, encode_nans=False)
        imputer = KNNImputer(n_neighbors=5, weights='distance')
        output = imputer.fit_transform(labeled_df)
        df_imputed = pd.DataFrame(output, columns=labeled_df.columns)
        df_final = copy.deepcopy(df_copy)
        for col in df_final:
            if df_final[col].isnull().any(): # only take the imputed column for columns that got missing values
                if col in dct_encoder:
                    # object columns
                    col_encoding = dct_encoder[col]
                    imputed_col = col_encoding.inverse_transform(list(round(df_imputed[col]).values.astype(int)))
                else:
                    if dtypes_dct[col] == pd.Int64Dtype():
                        # integer columns should not contain floats
                        imputed_col = pd.Series(list(round(df_imputed[col]).values.astype(int)))
                    else:
                        # float columns
                        imputed_col = df_imputed[col]
                df_final[col] = imputed_col
        for col in df_final:
            if pd.api.types.is_categorical_dtype(dtypes_dct[col]):
                df_final[col] = df_final[col].astype('category')
            else:
                df_final[col] = df_final[col].astype(dtypes_dct[col])
        return df_final
    else:
        for col in nan_cols:
            df_copy[col] = mv_method(df_copy, col, method)
    for col in df_copy:
        if pd.api.types.is_categorical_dtype(dtypes_dct[col]):
            df_copy[col] = df_copy[col].astype('category')
        else:
            df_copy[col] = df_copy[col].astype(dtypes_dct[col])
    return df_copy

## Duplicate instances <a class="anchor" id="duplicate_rows"></a>

### Detect

In [14]:
def detect_dup_row(df):
    ''' Detects duplicate rows'''
    dup_row_mask = df.duplicated()
    dup_row_idxs = df.index[dup_row_mask].to_list()
    print("Detected duplicate rows: ", dup_row_idxs)
    return dup_row_idxs

### Correct

In [15]:
def correct_dup_row(df, remove=True):
    ''' Corrects duplicate rows by dropping the duplictes except the first one'''
    if remove == True:
        duplicate_rows = detect_dup_row(df)
        new_df = df.drop(index=duplicate_rows)
    else:
        new_df = df
    return new_df

## Duplicate attributes <a class="anchor" id="duplicate_cols"></a>

### Detect

In [16]:
def detect_dup_col(df):
    ''' Detects duplicate columns'''
    dup_col_mask = df.T.duplicated()
    dup_col_names = df.columns[dup_col_mask].to_list()
    print("Detected duplicate columns: ", dup_col_names)
    return dup_col_names

### Correct

In [17]:
def remove_dup_col(df):
    ''' Corrects duplicate cols by dropping the duplicates except the first one'''
    duplicate_columns = detect_dup_col(df)
    new_df = df.drop(columns=duplicate_columns)
    return new_df

## Outliers values <a class="anchor" id="outlier_vals"></a>

### Detect

In [18]:
def detect_outlier_val(series, k=3):
    ''' Detects outlier values (only k=3 away cause the recommendation is to correct those and keep k=2)'''
    q25, q75 = percentile(series, 25), percentile(series,75)
    k_IQR = k * (q75 - q25)
    lower_bound, upper_bound = q25 - k_IQR, q75 + k_IQR
    outliers = set([val for val in series if val < lower_bound or val > upper_bound])
    return outliers

### Correct

In [57]:
def correct_outlier_val(df, feature_types, feature_types_dct, method="impute"):
    ''' Corrects outlier values by default using imputation'''
    outlier_dct = dict()
    int_cols = []
    for col, typ in df.dtypes.items(): # to convert int columns after imputation from float back to int
        if typ == 'int64':
            int_cols.append(col)
    for col, ft_type in feature_types_dct.items():
        if ft_type == 'numeric' and col in df.columns:
            outlier_vals = detect_outlier_val(df[col])
            if len(outlier_vals) != 0:
                outlier_dct[col] = outlier_vals
    print("Detected outlier values: ", outlier_dct)
    if method == "remove":
        idx_to_remove = []
        for col, outliers in outlier_dct.items():
            for idx, val in df[col].items():
                if val in outliers:
                    idx_to_remove.append(idx)
        df_imputed = df.drop(idx_to_remove)
    else:
        df_with_nans = copy.deepcopy(df)
        for col, vals in outlier_dct.items():
            for val in vals:
                indices = df_with_nans[df_with_nans[col] == val].index
                for idx in indices:
                    df_with_nans.loc[idx, col] = np.nan
        print(feature_types)
        df_imputed = impute_mv(df_with_nans, feature_types)
        for col in int_cols:
            df_imputed[col] = df_imputed[col].astype(int)
    return df_imputed

## Outlier instances <a class="anchor" id="outlier_rows"></a>

### Detect

In [22]:
def detect_outlier_row(df, threshold=0.8):
    ''' Detect outlier instances using Deepchecks'''
    if math.sqrt(len(df)) < 5: #minimum number of nearest neighbors
        nearest_neighbors_percent = 5/len(df)
        if nearest_neighbors_percent >= 1:
            nearest_neighbors_percent = 0.99
        if nearest_neighbors_percent == 0:
            nearest_neighbors_percent = 0.01
    else:
        nearest_neighbors_percent = round(((math.sqrt(len(df))) / len(df)), 2)
        if nearest_neighbors_percent == 0:
            nearest_neighbors_percent = 0.01
        if nearest_neighbors_percent >= 1:
            nearest_neighbors_percent = 0.99
    check = OutlierSampleDetection(n_to_show=10000, nearest_neighbors_percent=nearest_neighbors_percent, extent_parameter=3,  outlier_score_threshold=0.8, timeout=300)
    result = check.run(df)
    if result.display:
        output = result.display[1]
        outlier_indices = output[output['Outlier Probability Score'] > 0.8].index
        print("Detected outlier instances: ", outlier_indices)
    return outlier_indices

### Correct

In [23]:
def correct_outlier_row(df):
    ''' Correct Outlier instances by dropping them from the dataset'''
    outlier_indices = detect_outlier_row(df)
    new_df = df.drop(outlier_indices)
    return new_df

## Cryptic attribute names <a class="anchor" id="cryptic"></a>

### Classes and help functions

In [24]:
class CrypticIdentifier:
    """Module to identify any cryptic forms in a column header.
    Example usage: 
        identifier = CrypticIdentifier(vocab_file)
        identifier.iscryptic("newyorkcitytotalpopulation") --> False
        identifier.iscryptic("tot_revq4") --> True
    """
    def __init__(self, vocab_file=None, word_rank_file=None, k_whole=4, k_split=2):
        """
        Args:
            vocab_file (str, optional): json file containing the vocabulary. Defaults to None.
            k_whole (int, optional): length threshold for a whole string to be considered non-cryptic if it fails the first round of check (i.e.
            _iscryptic returns True). Defaults to 4.
            k_split (int, optional): length threshold for each word split (wordninja.split()) from the string to be considered non-cryptic, if the pre-split string fails the first round of check (i.e.
            _iscryptic returns True). Defaults to 2.
        """
        if vocab_file is not None:
            with open(vocab_file, "r") as fi:
                self.vocab = json.load(fi)
#                 print("#vocab={}".format(len(self.vocab)))
        else:
            self.vocab = None

        self.k_whole = k_whole
        self.k_split = k_split
        if word_rank_file is None:
            self.splitter = wordninja
        else:
            self.splitter = wordninja.LanguageModel(word_rank_file)
        self.lem = WordNetLemmatizer()
        

    def split_rm_punc(self, text: str) -> list:
        return re.sub(r'[^\w\s]', ' ', text).split()

    def separate_camel_case(self, text: str) -> list:
        return re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', text))

    def convert2base(self, text: str) -> str:
        return self.lem.lemmatize(text)

    def _split(self, text: str) -> list:
        text = text.replace('_', ' ')
        words = self.split_rm_punc(self.separate_camel_case(text))
        return words

    def _iscryptic(self, text: str) -> bool:
        words = self._split(text)
        if all([word.isnumeric() for word in words]):
            return True
        if self.vocab is None:
            self.vocab = nltk.corpus.wordnet.words('english')
        return any([self.convert2base(w.lower()) not in self.vocab for w in words])



    def doublecheck_cryptic(self, text: str) -> Tuple[bool, List[str]]:
        """Double-check whether a column header contains cryptic terms. For example in some cases where neither 
        delimiters between tokens nor camelcases is available

        Args:
            text (str): column header

        Returns:
            Tuple[
                    bool: whether header is cryptic
                    List[str]: splitted tokens from the header
                ]
        """

        #stopwords = nltk.corpus.stopwords.words('english')

        def split_check(words: List[str]) -> Tuple[bool, List[str]]:
            l_cryptic = []
            for ele in words:
                if ele.isdigit():
                    l_cryptic.append(False)
                ## Cornercases includes stopwords like "I", "for", etc.
                elif len(ele) < self.k_split: # and ele.lower() not in stopwords:
                    l_cryptic.append(True)
                ## Second round check
                else:
                    l_cryptic.append(self._iscryptic(ele))
            return any(l_cryptic), words
            
        if len(text) >= self.k_whole:
            if self._iscryptic(text):
                split = self.splitter.split(text)
                return split_check(split)            
            else:
                # return (False, self.splitter.split(text))
                return (False, self._split(text))
        else:
            with open('data/vocab/words.txt', 'r') as file:
                # Read the content of the file
                content = file.read()

                # Split the content into words based on whitespace
                vocabulair = content.split()

            # Print the list of words
            if text in vocabulair:
                return (False, [text])
            return (True, [text])

    def iscryptic(self, text: str) -> bool:
        return self.doublecheck_cryptic(text)[0]
    
    def split_results(self, text: str) -> List[str]:
        return self.doublecheck_cryptic(text)[1]

In [25]:
def query_cryp_cols(cryp_list, title, description, content_df):
    
    col_query = " | ".join(str(item) for item in cryp_list)
    
    if not content_df.empty:
        num_instances = len(content_df)
        content_list = [content_df.loc[idx].to_list() for idx in content_df.index]
        contents = "\n".join([" | ".join(str(item) for item in lst) for lst in content_list])
        content_bool = True
    else:
        content_bool = False
    
    if title == False:
        title_query = ""
    else:
        title_query = f"title: {title}"
    if description == False:
        desc_query = ""
    else:
        desc_query = f"description: {description}"
    if content_bool == False:
        content_query = ""
    else:
        content_query = f"contents of {num_instances} random instances:\n{contents}"
    
    if title == False and description == False and content_bool == False:
        subquery = ""
    else:
        subquery=f"""As abbreviations of column names from a table with characteristics:
{title_query}
{desc_query}
{content_query}
        """
    
    query = f"""As abbreviations of column names from a table, the column names c_name | pCd | dt stand for Customer Name | Product Code | Date.
{subquery}
The column names {col_query} stand for"""
    print(query)
    return query

### Detect

In [26]:
def detect_cryp_cols(df):
    ''' Detect cryptic column names in dataset'''
    identifier = CrypticIdentifier("./lookups/wordnet.json", "./lookups/wordninja_words_alpha.txt.gz")
    cryptic_cols = [col for col in df.columns if identifier.doublecheck_cryptic(col)[0]==True]
    return cryptic_cols

### Correct

In [27]:
def correct_cryp_cols(df, title=False, description=False, n_instances=0):
    ''' Corrects the cryptic column names using GPT-3.5'''
    # Set API key, uncomment to test
    client = OpenAI(api_key="") # add your own API key here
    
    # Identify the cryptic column names
    cryptic_cols = detect_cryp_cols(df)
    num_cols = len(cryptic_cols)
    
    if n_instances == 0:
        instances = pd.DataFrame()
    else:
        # Obtain n random instances of content for the cryptic columns
        sliced_df = df[cryptic_cols]
        indices = random.sample(range(0, len(sliced_df)), n_instances)
        instances = sliced_df.iloc[indices]
    
    # Generate the OpenAI query
    query = query_cryp_cols(cryptic_cols, title=title, description=description, content_df=instances)
    tokens_used = len(token_encoding.encode(query))
    print(tokens_used)
#     # 3 word ~= 4 tokens of OpenAI. We dont want too long column names (max. 4 words), which translates to ~ max 5 tokens per column name
#     max_token_len = num_cols * 4
    # Call OpenAI's GPT 3.5 Turbo LLM model to generate better column names
    completion = client.chat.completions.create(model="gpt-3.5-turbo",temperature=0.0,messages=[{"role": "user", "content": query}])
    message = completion.choices[0].message
    new_col_names = message.content
    return new_col_names, tokens_used

## Single value columns <a class="anchor" id="svs"></a>

### Detect

In [28]:
def detect_sv_col(df):
    '''Detects single value columns'''
    single_value_cols_list = []
    for col in df.columns:
        if df[col].nunique() <= 1:
            single_value_cols_list.append(col)
    print("Detected sv columns: ", single_value_cols_list)
    return single_value_cols_list

### Correct

In [29]:
def correct_sv_col(df, keep=False):
    ''' Corrects single value columns by removing them'''
    if not keep:
        single_value_cols_list = detect_sv_col(df)
        df = df.drop(columns=single_value_cols_list)
    return df

## Mixed data type columns <a class="anchor" id="mixed"></a>

### Classes and Help functions

In [31]:
def convert_mix_to_nan(df, mix_dct):
    ''' Converts the minority data type in mixed data columns to NaN values'''
    for col, dct in mix_dct.items():
        if dct['numbers'] < dct['strings']:
            #numbers to np.nan
            for idx, val in df[col].items():
                if isinstance(val, str) and val.isdigit():
                    df.loc[idx, col] = np.nan
                elif isinstance(val, (int, float)):
                    df.loc[idx, col] = np.nan
        else:
            #strings to np.nan
            for idx, val in df[col].items():
                if isinstance(val, str) and not val.isdigit():
                    df.loc[idx, col] = np.nan
    return df

### Detect

In [35]:
def detect_mixed_data(df):
    ''' Detects mixed data types using deepchecks'''
    result = MixedDataTypes(n_top_columns=10000).run(df)
    mixed_dct = result.value
    delete_cols = []
    for col, mixed in mixed_dct.items():
        if len(mixed) == 0:
            delete_cols.append(col)
    for col in delete_cols:
        del mixed_dct[col]
    print("Detected mixed data: ", mixed_dct)
    return mixed_dct

### Correct

In [37]:
def correct_mixed_data(df, feature_types):
    ''' Corrects mixed data using deepchecks'''
    df_copy = copy.deepcopy(df)
    mixed_dct = detect_mixed_data(df_copy)
    df_with_nans = convert_mix_to_nan(df_copy, mixed_dct)
    df_imputed = impute_mv(df_with_nans, feature_types)
    return df_imputed

## Mislabeled labels <a class="anchor" id="mislabels"></a>

### Detect

In [39]:
def detect_mislabels(df, target):
    ''' Detects Incorrect labels using Cleanlab'''
    model_XGBC = XGBClassifier(tree_method="hist", enable_categorical=True)
    df_numeric, encoding = categorical_to_label(df)
    X = df_numeric.drop(columns=[target])
    y = df_numeric[target]
    
    stratified_splits = StratifiedKFold(n_splits=5)
    pred_probs = cross_val_predict(model_XGBC, X, y, cv=stratified_splits, method='predict_proba')
    cl = cleanlab.classification.CleanLearning()
    df_label_issues = cl.find_label_issues(X=None, labels=y, pred_probs=pred_probs)
    print("Detected mislabels: ", df_label_issues[df_label_issues['is_label_issue'] == True])
    return df_label_issues, encoding

In [40]:
def correct_mislabels(df, target, ft_types_dct):
    ''' Corrects incorrect labels and converts them to correct label'''
    df_copy = copy.deepcopy(df)
    if ft_types_dct[target] == 'categorical':
        df_label_issues, encoding = detect_mislabels(df_copy, target)
        df_label_issues['predicted_label'] = encoding[target].inverse_transform(df_label_issues['predicted_label'].to_list())
        dicty = {idx : df_label_issues.loc[idx,'predicted_label'] for idx in df_label_issues[df_label_issues['is_label_issue'] == True].index}
        col_idx = [col_idx for col_idx in range(len(df_copy.columns)) if df_copy.columns[col_idx] == target][0]
        for idx in dicty.keys():
            df_copy.iloc[idx,col_idx] = dicty[idx]
    return df_copy

## String mismatch <a class="anchor" id="mismatches"></a>

### Detect

In [41]:
def detect_mismatch(df, target):
    ''' Detects string mismatches using deepchecks'''
    result = StringMismatch(n_top_columns=10000).run(df)
    dct_mismatch = result.value['columns']
    print(dct_mismatch)
    delete_cols = []
    for col, mismatches in dct_mismatch.items():
        if len(mismatches) == 0:
            delete_cols.append(col)
    for col in delete_cols:
        print(col)
        del dct_mismatch[col]
    if target in dct_mismatch:
        del dct_mismatch[target]
    df_dct = dict()
    for col, data in dct_mismatch.items():
        for base, var in data.items():
            new_df = pd.DataFrame(var)
            df_dct[(col, base)] = new_df
    print("Detected mismatches: ", dct_mismatch)
    return df_dct, dct_mismatch

### Correct

In [42]:
def correct_mismatch(df, target):
    ''' Corrects the string mismatches by converting them to base form'''
    df_copy = copy.deepcopy(df)
    print(df_copy)
    dct_df, mismatch_dct = detect_mismatch(df_copy, target)
    for col, mismatches in mismatch_dct.items():
        variations = []
        for base, var_dct_list in mismatches.items():
            for var in var_dct_list:
                variations.append(var['variant'])
        variation_idx = df_copy[df_copy[col].isin(variations)].index
        df_copy.loc[variation_idx, col] = base
    return df_copy

## Tests

### Missing values

In [43]:
detect_mv(df_synthetic)

[['gender', 'nan', 11]]

In [44]:
detect_dmv(df_synthetic)

[['workclass', 'Variation@', 3],
 ['education', '?', 7],
 ['education', '0', 3],
 ['education', '-', 3]]

### Duplicates

In [45]:
detect_dup_row(df_synthetic)

Detected duplicate rows:  [20, 25, 34, 43, 46, 47]


[20, 25, 34, 43, 46, 47]

In [46]:
detect_dup_col(df_synthetic)

Detected duplicate columns:  ['duplicate column']


['duplicate column']

### Outliers

In [47]:
for col, ft_type in feature_types_dct.items():
    if ft_type == 'numeric':
        print(col)
        print(detect_outlier_val(df_synthetic[col]))

age
{128, 167, 137, 139, 143}
fnlwgt
set()
duplicate column
{128, 167, 137, 139, 143}


In [48]:
detect_outlier_row(df_synthetic)

deepchecks - WARNING - Received a "pandas.DataFrame" instance. It is recommended to pass a "deepchecks.tabular.Dataset" instance by initializing it with the data and metadata, for example by doing "Dataset(dataframe, label=label, cat_features=cat_features)"
deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 6 categorical features were inferred.: gender, native-country, income, high_label_correlation, redundant SV column, mixed_data_types


Detected outlier instances:  Index([], dtype='int64')


Index([], dtype='int64')

### Cryptic column names

In [49]:
detect_cryp_cols(df_synthetic)

['fnlwgt', 'redundant SV column']

### Single value columns

In [50]:
detect_sv_col(df_synthetic)

Detected sv columns:  ['redundant SV column']


['redundant SV column']

### Mixed data

In [51]:
detect_mixed_data(df_synthetic)

deepchecks - WARNING - Received a "pandas.DataFrame" instance. It is recommended to pass a "deepchecks.tabular.Dataset" instance by initializing it with the data and metadata, for example by doing "Dataset(dataframe, label=label, cat_features=cat_features)"
deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 6 categorical features were inferred.: gender, native-country, income, high_label_correlation, redundant SV column, mixed_data_types


Detected mixed data:  {'education': {'strings': 0.9454545454545454, 'numbers': 0.05454545454545454, 'strings_examples': {'Bachelors', 'Masters', 'HS-grad'}, 'numbers_examples': {0}}, 'mixed_data_types': {'strings': 0.2727272727272727, 'numbers': 0.7272727272727273, 'strings_examples': {'MD2', 'MD3', 'MD1'}, 'numbers_examples': {1, 2, 3}}}


{'education': {'strings': 0.9454545454545454,
  'numbers': 0.05454545454545454,
  'strings_examples': {'Bachelors', 'HS-grad', 'Masters'},
  'numbers_examples': {0}},
 'mixed_data_types': {'strings': 0.2727272727272727,
  'numbers': 0.7272727272727273,
  'strings_examples': {'MD1', 'MD2', 'MD3'},
  'numbers_examples': {1, 2, 3}}}

### Mislabeled labels

In [59]:
detect_mislabels(df_synthetic, target)

Detected mislabels:      is_label_issue  label_quality  given_label  predicted_label
12            True       0.279960            0                1
37            True       0.168822            0                1
45            True       0.385885            0                1
50            True       0.009794            1                0


(    is_label_issue  label_quality  given_label  predicted_label
 0            False       0.942201            0                0
 1            False       0.949764            0                0
 2            False       0.996256            0                0
 3            False       0.921297            0                0
 4            False       0.542952            0                0
 5            False       0.985606            0                0
 6            False       0.982427            0                0
 7            False       0.981859            0                0
 8            False       0.990919            0                0
 9            False       0.840710            0                0
 10           False       0.959199            0                0
 11           False       0.992264            0                0
 12            True       0.279960            0                1
 13           False       0.932174            0                0
 14           False      

### String mismatches

In [61]:
detect_mismatch(df_synthetic, target)

deepchecks - WARNING - Received a "pandas.DataFrame" instance. It is recommended to pass a "deepchecks.tabular.Dataset" instance by initializing it with the data and metadata, for example by doing "Dataset(dataframe, label=label, cat_features=cat_features)"
deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 6 categorical features were inferred.: gender, native-country, income, high_label_correlation, redundant SV column, mixed_data_types


{'workclass': {'variation': [{'variant': 'VARIATION', 'count': 1, 'percent': 0.01818181818181818}, {'variant': 'VARiation!', 'count': 1, 'percent': 0.01818181818181818}, {'variant': 'vaRiAtIoN', 'count': 3, 'percent': 0.05454545454545454}, {'variant': 'variation', 'count': 2, 'percent': 0.03636363636363636}, {'variant': 'Variation@', 'count': 3, 'percent': 0.05454545454545454}]}, 'education': {}, 'gender': {}, 'native-country': {}, 'income': {'50k': [{'variant': '<=50K', 'count': 44, 'percent': 0.8}, {'variant': '>50K', 'count': 11, 'percent': 0.2}]}, 'high_label_correlation': {}, 'redundant SV column': {}, 'mixed_data_types': {}}
education
gender
native-country
high_label_correlation
redundant SV column
mixed_data_types
Detected mismatches:  {'workclass': {'variation': [{'variant': 'VARIATION', 'count': 1, 'percent': 0.01818181818181818}, {'variant': 'VARiation!', 'count': 1, 'percent': 0.01818181818181818}, {'variant': 'vaRiAtIoN', 'count': 3, 'percent': 0.05454545454545454}, {'varia

({('workclass',
   'variation'):       variant  count   percent
  0   VARIATION      1  0.018182
  1  VARiation!      1  0.018182
  2   vaRiAtIoN      3  0.054545
  3   variation      2  0.036364
  4  Variation@      3  0.054545},
 {'workclass': {'variation': [{'variant': 'VARIATION',
     'count': 1,
     'percent': 0.01818181818181818},
    {'variant': 'VARiation!', 'count': 1, 'percent': 0.01818181818181818},
    {'variant': 'vaRiAtIoN', 'count': 3, 'percent': 0.05454545454545454},
    {'variant': 'variation', 'count': 2, 'percent': 0.03636363636363636},
    {'variant': 'Variation@', 'count': 3, 'percent': 0.05454545454545454}]}})

# Downstream model performance

## Gather datasets

In [62]:
import openml
import sortinghatinf
openml.datasets.list_datasets(output_format="dataframe")

,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
2,2,anneal,1,1,active,ARFF,684.0,7.0,8.0,5.0,39.0,898.0,898.0,22175.0,6.0,33.0
3,3,kr-vs-kp,1,1,active,ARFF,1669.0,3.0,1527.0,2.0,37.0,3196.0,0.0,0.0,0.0,37.0
4,4,labor,1,1,active,ARFF,37.0,3.0,20.0,2.0,17.0,57.0,56.0,326.0,8.0,9.0
5,5,arrhythmia,1,1,active,ARFF,245.0,13.0,2.0,13.0,280.0,452.0,384.0,408.0,206.0,74.0
6,6,letter,1,1,active,ARFF,813.0,26.0,734.0,26.0,17.0,20000.0,0.0,0.0,16.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46254,46254,Diabetes_Dataset,2,39999,active,arff,NaN,NaN,NaN,NaN,9.0,768.0,0.0,0.0,9.0,0.0
46255,46255,Student_Performance_Dataset,1,39999,active,arff,NaN,NaN,NaN,NaN,15.0,2392.0,0.0,0.0,15.0,0.0
46258,46258,sonar,2,43180,active,arff,NaN,NaN,NaN,NaN,61.0,207.0,0.0,0.0,60.0,0.0
46259,46259,Electricity-hourly,1,30703,active,arff,NaN,NaN,NaN,NaN,319.0,26305.0,0.0,0.0,317.0,1.0


In [77]:
# We have chosen by hand 10 regression and 10 classification datasets
regression_datasets = [8, 204, 210, 560, 491, 566, 189, 673, 639, 41700]
classification_datasets = [44, 38, 458, 1053, 1050, 2, 3, 5, 31, 1464]
complete_did_list = regression_datasets + classification_datasets

## Create clean version of each dataset

In [63]:
def full_cleaner(df, feature_types, feature_types_dct, target):
    df = correct_dup_row(df).reset_index(drop=True) #dup 1
    df = remove_dup_col(df) # dup 2
    df = correct_outlier_row(df) # out 1
    df = correct_sv_col(df) # sv
    df = correct_mismatch(df, target) # mm
    df = correct_mislabels(df, target, feature_types_dct) #mislabels
    df = impute_mv(df, feature_types) # mv
    df = correct_outlier_val(df, feature_types, feature_types_dct) # out 2
    df = correct_mixed_data(df, feature_types) # mix
    return df

## Final Data Quality Experiment

In [75]:
def evaluate_datasets(did_list):
    ''' Evaluates for every dataset the performance on the dirty and clean dataset'''
    ds_perfs = dict()
    
    for did in did_list:
        dataset = openml.datasets.get_dataset(did)
        X, y, categorical_indicator, attribute_names = dataset.get_data(
            target=dataset.default_target_attribute
        )
        y = pd.DataFrame(y)
        df = pd.concat([X, y], axis=1)
        ft_types = sortinghatinf.get_sortinghat_types(df)
        ft_types_dct = {col: typ for col, typ in zip(df.columns, ft_types)}
        
        clean_df = full_cleaner(df, ft_types, ft_types_dct, df.columns[-1])
        
        task = 'classification' if ft_types[-1] == 'categorical' else 'regression'

        new_df = df.dropna()
        if len(new_df) < 0.2 * len(df):
            new_df = df.dropna(axis=1)
        
        X_dirty = new_df.drop(columns=[new_df.columns[-1]])
        y_dirty = new_df[new_df.columns[-1]]
        enc_X_dirty, enc_dirty = categorical_to_label(X_dirty)
        
        X_clean = clean_df.drop(columns=[df.columns[-1]])
        y_clean = clean_df[df.columns[-1]]
        enc_X_clean, enc_clean = categorical_to_label(X_clean)

        ds_perfs[did] = evaluate_performance(enc_X_dirty, y_dirty, enc_X_clean, y_clean, task)

        with open('data/data/performances_datasets_dirty_clean.pkl', 'wb') as handle:
            pickle.dump(ds_perfs, handle)
    
    return ds_perfs

def evaluate_performance(X_dirty, y_dirty, X_clean, y_clean, task):
    ''' Calculates the performance of the 5 different models using cross validation'''
    models = {
        'RF': RandomForestClassifier() if task == 'classification' else RandomForestRegressor(),
        'LR': LogisticRegression() if task == 'classification' else LinearRegression(),
        'KNN': KNeighborsClassifier() if task == 'classification' else KNeighborsRegressor(),
        'DT': DecisionTreeClassifier() if task == 'classification' else DecisionTreeRegressor(),
        'GB': GradientBoostingClassifier() if task == 'classification' else GradientBoostingRegressor()
    }
    
    metrics = {}
    for model_name, model in models.items():
        metrics[model_name] = {
            'dirty': calculate_performance(X_dirty, y_dirty, task, model),
            'clean': calculate_performance(X_clean, y_clean, task, model)
        }
    
    return metrics

def calculate_performance(X, y, task, model):
    ''' Function for cross validation and what performance measure to use'''
    if task == 'classification':
        cv = StratifiedKFold(n_splits=5)
        scoring = ['f1_weighted', 'accuracy']
    else:
        cv = KFold(n_splits=5)
        scoring = ['neg_mean_absolute_error', 'neg_root_mean_squared_error']
    
    scores = cross_validate(model, X, y, cv=cv, scoring=scoring)
    
    if task == 'classification':
        performance = {
            'F1': scores['test_f1_weighted'].mean(),
            'Accuracy': scores['test_accuracy'].mean(),
        }
    else:
        performance = {
            'MAE': -scores['test_neg_mean_absolute_error'].mean(),
            'RMSE': -scores['test_neg_root_mean_squared_error'].mean()}
    
    return performance

In [82]:
# Load in the saved performances for all datasets
with open("data/performances_datasets_dirty_clean.pkl", 'rb') as f:
    downstream_perfs_dct = pickle.load(f)
downstream_perfs_dct

{8: {'RF': {'dirty': {'MAE': 2.8779565217391303, 'RMSE': 3.4075470512183026},
   'clean': {'MAE': 2.8728582089552237, 'RMSE': 3.4040742399517443}},
  'LR': {'dirty': {'MAE': 2.8014647951133584, 'RMSE': 3.364155310843141},
   'clean': {'MAE': 2.7335902830502987, 'RMSE': 3.2783025527080403}},
  'KNN': {'dirty': {'MAE': 3.0234782608695654, 'RMSE': 3.6525838429736104},
   'clean': {'MAE': 3.103617208077261, 'RMSE': 3.7102125253265057}},
  'DT': {'dirty': {'MAE': 3.6869565217391305, 'RMSE': 4.689735253112677},
   'clean': {'MAE': 3.517866549604917, 'RMSE': 4.354162113760905}},
  'GB': {'dirty': {'MAE': 2.88346309086793, 'RMSE': 3.4530471410787813},
   'clean': {'MAE': 2.901515597717368, 'RMSE': 3.4566025860491605}}},
 204: {'RF': {'dirty': {'MAE': 42.80966101694916, 'RMSE': 54.75768759808791},
   'clean': {'MAE': 40.26471147540983, 'RMSE': 50.32794662629982}},
  'LR': {'dirty': {'MAE': 40.24055142704408, 'RMSE': 52.49827100690286},
   'clean': {'MAE': 37.92614771342156, 'RMSE': 48.115418906

In [68]:
# Create a dataframe with all performances for all datasets for all models for all measures for classification task
classification = pd.DataFrame(columns = [
    'dirty_rf_f1', 'dirty_rf_acc',
    'dirty_lr_f1', 'dirty_lr_acc',
    'dirty_knn_f1', 'dirty_knn_acc',
    'dirty_dt_f1', 'dirty_dt_acc',
    'dirty_gb_f1', 'dirty_gb_acc',
    'clean_rf_f1', 'clean_rf_acc',
    'clean_lr_f1', 'clean_lr_acc',
    'clean_knn_f1', 'clean_knn_acc',
    'clean_dt_f1', 'clean_dt_acc',
    'clean_gb_f1', 'clean_gb_acc'
])
for key, value in downstream_perfs_dct.items():
    if key in classification_datasets:
        dirty = []
        clean = []
        for model, perfs in value.items():
            dirty_f1 = perfs['dirty']['F1']
            dirty_acc = perfs['dirty']['Accuracy']
            dirty.append(round(dirty_f1,2))
            dirty.append(round(dirty_acc,2))
            clean_f1 = perfs['clean']['F1']
            clean_acc = perfs['clean']['Accuracy']
            clean.append(round(clean_f1,2))
            clean.append(round(clean_acc,2))
        all_perfs = dirty + clean
        classification.loc[key] = all_perfs
classification

,dirty_rf_f1,dirty_rf_acc,dirty_lr_f1,dirty_lr_acc,dirty_knn_f1,dirty_knn_acc,dirty_dt_f1,dirty_dt_acc,dirty_gb_f1,dirty_gb_acc,clean_rf_f1,clean_rf_acc,clean_lr_f1,clean_lr_acc,clean_knn_f1,clean_knn_acc,clean_dt_f1,clean_dt_acc,clean_gb_f1,clean_gb_acc
44,0.93,0.93,0.90,0.90,0.77,0.77,0.88,0.88,0.93,0.93,0.90,0.90,0.85,0.85,0.74,0.74,0.84,0.84,0.89,0.89
38,0.91,0.94,0.91,0.94,0.91,0.93,0.91,0.94,0.91,0.94,0.98,0.98,0.96,0.96,0.91,0.93,0.98,0.98,0.98,0.99
458,0.98,0.98,0.99,0.99,0.98,0.98,0.82,0.83,0.93,0.92,0.94,0.94,0.91,0.91,0.93,0.94,0.89,0.90,0.93,0.93
1053,0.76,0.79,0.73,0.76,0.74,0.76,0.72,0.71,0.75,0.80,0.86,0.87,0.81,0.82,0.82,0.84,0.83,0.82,0.86,0.87
1050,0.88,0.90,0.87,0.90,0.84,0.88,0.86,0.85,0.88,0.90,0.91,0.93,0.90,0.92,0.89,0.91,0.91,0.91,0.92,0.93
2,0.89,0.90,0.69,0.76,0.80,0.82,0.87,0.88,0.90,0.91,1.00,1.00,0.75,0.82,0.89,0.90,1.00,1.00,1.00,1.00
3,0.94,0.94,0.93,0.93,0.78,0.78,0.98,0.98,0.94,0.94,0.95,0.95,0.93,0.93,0.78,0.78,0.98,0.98,0.95,0.95
5,0.65,0.72,0.61,0.62,0.51,0.62,0.56,0.56,0.65,0.69,0.69,0.75,0.68,0.70,0.51,0.62,0.68,0.68,0.72,0.73
31,0.74,0.76,0.69,0.71,0.62,0.65,0.69,0.68,0.74,0.75,0.84,0.84,0.73,0.75,0.64,0.67,0.75,0.75,0.83,0.83
1464,0.64,0.67,0.71,0.77,0.64,0.67,0.60,0.61,0.67,0.71,0.60,0.62,0.70,0.73,0.58,0.60,0.60,0.61,0.66,0.69


In [69]:
classification.sort_index()

,dirty_rf_f1,dirty_rf_acc,dirty_lr_f1,dirty_lr_acc,dirty_knn_f1,dirty_knn_acc,dirty_dt_f1,dirty_dt_acc,dirty_gb_f1,dirty_gb_acc,clean_rf_f1,clean_rf_acc,clean_lr_f1,clean_lr_acc,clean_knn_f1,clean_knn_acc,clean_dt_f1,clean_dt_acc,clean_gb_f1,clean_gb_acc
2,0.89,0.90,0.69,0.76,0.80,0.82,0.87,0.88,0.90,0.91,1.00,1.00,0.75,0.82,0.89,0.90,1.00,1.00,1.00,1.00
3,0.94,0.94,0.93,0.93,0.78,0.78,0.98,0.98,0.94,0.94,0.95,0.95,0.93,0.93,0.78,0.78,0.98,0.98,0.95,0.95
5,0.65,0.72,0.61,0.62,0.51,0.62,0.56,0.56,0.65,0.69,0.69,0.75,0.68,0.70,0.51,0.62,0.68,0.68,0.72,0.73
31,0.74,0.76,0.69,0.71,0.62,0.65,0.69,0.68,0.74,0.75,0.84,0.84,0.73,0.75,0.64,0.67,0.75,0.75,0.83,0.83
38,0.91,0.94,0.91,0.94,0.91,0.93,0.91,0.94,0.91,0.94,0.98,0.98,0.96,0.96,0.91,0.93,0.98,0.98,0.98,0.99
44,0.93,0.93,0.90,0.90,0.77,0.77,0.88,0.88,0.93,0.93,0.90,0.90,0.85,0.85,0.74,0.74,0.84,0.84,0.89,0.89
458,0.98,0.98,0.99,0.99,0.98,0.98,0.82,0.83,0.93,0.92,0.94,0.94,0.91,0.91,0.93,0.94,0.89,0.90,0.93,0.93
1050,0.88,0.90,0.87,0.90,0.84,0.88,0.86,0.85,0.88,0.90,0.91,0.93,0.90,0.92,0.89,0.91,0.91,0.91,0.92,0.93
1053,0.76,0.79,0.73,0.76,0.74,0.76,0.72,0.71,0.75,0.80,0.86,0.87,0.81,0.82,0.82,0.84,0.83,0.82,0.86,0.87
1464,0.64,0.67,0.71,0.77,0.64,0.67,0.60,0.61,0.67,0.71,0.60,0.62,0.70,0.73,0.58,0.60,0.60,0.61,0.66,0.69


In [70]:
# Calculate average scores per dataset for dirty datasets
for idx in classification.index:
    print(f"F1 {idx}: ", round(mean(classification.loc[idx][['dirty_rf_f1', 'dirty_lr_f1', 'dirty_knn_f1', 'dirty_dt_f1', 'dirty_gb_f1']]),2))
    print(f"Acc {idx}: ", round(mean(classification.loc[idx][['dirty_rf_acc', 'dirty_lr_acc', 'dirty_knn_acc', 'dirty_dt_acc', 'dirty_gb_acc']]),2))

F1 44:  0.88
Acc 44:  0.88
F1 38:  0.91
Acc 38:  0.94
F1 458:  0.94
Acc 458:  0.94
F1 1053:  0.74
Acc 1053:  0.76
F1 1050:  0.87
Acc 1050:  0.89
F1 2:  0.83
Acc 2:  0.85
F1 3:  0.91
Acc 3:  0.91
F1 5:  0.6
Acc 5:  0.64
F1 31:  0.7
Acc 31:  0.71
F1 1464:  0.65
Acc 1464:  0.69


In [71]:
# Calculate average scores per dataset for clean datasets
for idx in classification.index:
    print(f"F1 {idx}: ", round(mean(classification.loc[idx][['clean_rf_f1', 'clean_lr_f1', 'clean_knn_f1', 'clean_dt_f1', 'clean_gb_f1']]), 2))
    print(f"Acc {idx}: ", round(mean(classification.loc[idx][['clean_rf_acc', 'clean_lr_acc', 'clean_knn_acc', 'clean_dt_acc', 'clean_gb_acc']]), 2))

F1 44:  0.84
Acc 44:  0.84
F1 38:  0.96
Acc 38:  0.97
F1 458:  0.92
Acc 458:  0.92
F1 1053:  0.84
Acc 1053:  0.84
F1 1050:  0.91
Acc 1050:  0.92
F1 2:  0.93
Acc 2:  0.94
F1 3:  0.92
Acc 3:  0.92
F1 5:  0.66
Acc 5:  0.7
F1 31:  0.76
Acc 31:  0.77
F1 1464:  0.63
Acc 1464:  0.65


In [124]:
# Store the performances in an excel file
classification.to_excel('data/perfs_cv_class.xlsx')

C:\Users\sande\AppData\Local\Temp\ipykernel_9812\4218847766.py:1: UserWarning:

Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).



In [72]:
# Create a dataframe with all performances for all datasets for all models for all measures for regression task
regression = pd.DataFrame(columns = [
    'dirty_rf_mae', 'dirty_rf_rmse',
    'dirty_lr_mae', 'dirty_lr_rmse',
    'dirty_knn_mae', 'dirty_knn_rmse',
    'dirty_dt_mae', 'dirty_dt_rmse',
    'dirty_gb_mae', 'dirty_gb_rmse',
    'clean_rf_mae', 'clean_rf_rmse',
    'clean_lr_mae', 'clean_lr_rmse',
    'clean_knn_mae', 'clean_knn_rmse',
    'clean_dt_mae', 'clean_dt_rmse',
    'clean_gb_mae', 'clean_gb_rmse'
])

for key, value in downstream_perfs_dct.items():
    if key in regression_datasets:
        dirty = []
        clean = []
        for model, perfs in value.items():
            dirty_mae = perfs['dirty']['MAE']
            dirty_rmse = perfs['dirty']['RMSE']
            dirty.append(round(dirty_mae, 2))
            dirty.append(round(dirty_rmse, 2))
            clean_mae = perfs['clean']['MAE']
            clean_rmse = perfs['clean']['RMSE']
            clean.append(round(clean_mae, 2))
            clean.append(round(clean_rmse, 2))
        all_perfs = dirty + clean
        regression.loc[key] = all_perfs
regression

,dirty_rf_mae,dirty_rf_rmse,dirty_lr_mae,dirty_lr_rmse,dirty_knn_mae,dirty_knn_rmse,dirty_dt_mae,dirty_dt_rmse,dirty_gb_mae,dirty_gb_rmse,clean_rf_mae,clean_rf_rmse,clean_lr_mae,clean_lr_rmse,clean_knn_mae,clean_knn_rmse,clean_dt_mae,clean_dt_rmse,clean_gb_mae,clean_gb_rmse
8,2.88,3.41,2.80,3.36,3.02,3.65,3.69,4.69,2.88,3.45,2.87,3.40,2.73,3.28,3.10,3.71,3.52,4.35,2.90,3.46
204,42.81,54.76,40.24,52.50,43.32,55.64,56.38,71.98,45.00,57.41,40.26,50.33,37.93,48.12,41.37,51.80,56.68,69.69,42.89,53.33
210,0.31,0.49,0.26,0.39,0.35,0.54,0.37,0.59,0.30,0.47,0.25,0.39,0.27,0.42,0.31,0.47,0.32,0.49,0.28,0.40
560,0.49,1.48,0.55,1.19,5.00,6.00,0.78,2.23,0.48,1.36,0.49,1.46,0.59,1.23,4.98,5.99,0.76,2.30,0.49,1.35
491,0.66,0.90,0.60,0.77,0.63,0.86,0.72,1.13,0.69,0.98,0.66,0.93,0.60,0.79,0.65,0.90,0.71,1.14,0.72,1.02
566,202.80,685.51,244.50,624.64,256.01,748.67,263.52,1015.13,214.32,727.84,12.96,17.34,13.78,18.90,15.45,20.80,19.43,27.40,14.78,20.63
189,0.11,0.14,0.16,0.20,0.09,0.12,0.16,0.21,0.14,0.18,0.11,0.14,0.16,0.20,0.09,0.12,0.16,0.21,0.14,0.18
673,0.42,0.52,0.41,0.51,0.42,0.53,0.55,0.69,0.43,0.54,0.41,0.52,0.41,0.51,0.42,0.53,0.57,0.71,0.42,0.54
639,0.67,0.85,0.97,1.16,0.72,0.92,0.88,1.21,0.59,0.79,0.67,0.86,0.97,1.16,0.72,0.92,0.88,1.16,0.60,0.79
41700,281.51,486.13,1274.65,1467.99,1403.92,1690.89,353.51,776.57,262.55,467.15,275.64,472.80,1259.52,1523.58,1345.59,1709.48,388.57,805.77,260.27,456.03


In [140]:
# Store the performances in an excel file
regression.to_excel("data/perfs_cv_regr.xlsx")

C:\Users\sande\AppData\Local\Temp\ipykernel_9812\2389794404.py:1: UserWarning:

Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).



In [78]:
# Calculate average scores per dataset for clean datasets
for idx in regression.index:
    print(f"MAE {idx}: ", round(mean(regression.loc[idx][['clean_rf_mae', 'clean_lr_mae', 'clean_knn_mae', 'clean_dt_mae', 'clean_gb_mae']]), 2))
    print(f"RMSE {idx}: ", round(mean(regression.loc[idx][['clean_rf_rmse', 'clean_lr_rmse', 'clean_knn_rmse', 'clean_dt_rmse', 'clean_gb_rmse']]), 2))

MAE 8:  3.02
RMSE 8:  3.64
MAE 204:  43.83
RMSE 204:  54.65
MAE 210:  0.29
RMSE 210:  0.43
MAE 560:  1.46
RMSE 560:  2.47
MAE 491:  0.67
RMSE 491:  0.96
MAE 566:  15.28
RMSE 566:  21.01
MAE 189:  0.13
RMSE 189:  0.17
MAE 673:  0.45
RMSE 673:  0.56
MAE 639:  0.77
RMSE 639:  0.98
MAE 41700:  705.92
RMSE 41700:  993.53


In [79]:
# Calculate average scores per dataset for dirty datasets
for idx in regression.index:
    print(f"MAE {idx}: ", round(mean(regression.loc[idx][['dirty_rf_mae', 'dirty_lr_mae', 'dirty_knn_mae', 'dirty_dt_mae', 'dirty_gb_mae']]), 2))
    print(f"RMSE {idx}: ", round(mean(regression.loc[idx][['dirty_rf_rmse', 'dirty_lr_rmse', 'dirty_knn_rmse', 'dirty_dt_rmse', 'dirty_gb_rmse']]), 2))


MAE 8:  3.05
RMSE 8:  3.71
MAE 204:  45.55
RMSE 204:  58.46
MAE 210:  0.32
RMSE 210:  0.5
MAE 560:  1.46
RMSE 560:  2.45
MAE 491:  0.66
RMSE 491:  0.93
MAE 566:  236.23
RMSE 566:  760.36
MAE 189:  0.13
RMSE 189:  0.17
MAE 673:  0.45
RMSE 673:  0.56
MAE 639:  0.77
RMSE 639:  0.99
MAE 41700:  715.23
RMSE 41700:  977.75
